In [1]:
# import matplotlib
# matplotlib.use('Agg')
# import matplotlib.pyplot as plt
# import matplotlib.mlab as mlab
# from matplotlib import gridspec
# import matplotlib.dates as mdates

import tkp.db
import tkp.config
import logging
import csv
import time
import sys
import glob
import itertools


# import healpy as hp
import numpy as np
import pandas as pd
import scipy as sp
import datetime
import os

from astropy import units as u
from astropy.coordinates import SkyCoord, match_coordinates_sky
from astropy.io import fits
from astropy.wcs import WCS

# import numbers
# import math
# import scipy
# from scipy.stats import norm
# from scipy.stats import sem
# from scipy import linspace
# from scipy import pi,sqrt,exp
# from scipy.special import erf

# import pylab

# from matplotlib.ticker import NullFormatter
# from matplotlib.font_manager import FontProperties



RuntimeError: module compiled against API version 0xc but this version of numpy is 0xa

/afhome/kuiack/newtrap/local/lib/python2.7/site-packages/pyfits/__init__.py:22: PyFITSDeprecationWarning: PyFITS is deprecated, please use astropy.io.fits
  PyFITSDeprecationWarning)  # noqa


In [2]:
def nsf(num, n=1):
    """n-Significant Figures"""
    numstr = ("{0:.%ie}" % (n-1)).format(num)
    return float(numstr)

def num_err(num, err, n=1):
    '''Return number rounded based on error'''
    return np.around(num,int(-(np.floor(np.log10(nsf(err,n=n)))))), nsf(err,n=n)

def clip(data, sigma=3):
    """Remove all values above a threshold from the array.
    Uses iterative clipping at sigma value until nothing more is getting clipped.
    Args:
        data: a numpy array
    """
    data = data[np.isfinite(data)]
    raveled = data.ravel()
    median = np.median(raveled)
    std = np.nanstd(raveled)
    newdata = raveled[np.abs(raveled-median) <= sigma*std]
    if len(newdata) and len(newdata) != len(raveled):
        return clip(newdata, sigma)
    else:
        return newdata
    
def distSquared(p0, p1):
    distance  = np.sqrt((p0[0] - p1[0,:])**2 + (p0[1] - p1[1,:])**2)
    if np.min(distance) < 3.0:
        return np.where(distance == np.min(distance))[0]
    else:
        return None



def pol2cart(rho, phi):
    x = rho * np.cos(phi)
    y = rho * np.sin(phi)
    return(x, y)


def run_query(transients_query, dbname, dataset_id, engine, host, port, user, pword):
    tkp.db.Database(
        database=dbname, user=user, password=pword,
        engine=engine, host=host, port=port
    )

    cursor = tkp.db.execute(transients_query, (dataset_id,))
    transients = tkp.db.generic.get_db_rows_as_dicts(cursor)
    return transients


In [3]:
engine = 'postgresql'
host = 'ads001'
port = 5432
user = 'mkuiack'
password = 'Morecomplicatedpass1234'

query_loglevel = logging.WARNING  # Set to INFO to see queries, otherwise WARNING


In [4]:
vlssr = pd.read_csv("/home/kuiack/VLSSr.tsv", comment="#", delimiter="\t")
# tgss = pd.read_csv("/home/kuiack/TGSSADR1_7sigma_catalog.tsv", delimiter="\t")

aart = pd.read_csv("/home/kuiack/AARTFAAC_catalogue.csv")
ateam = {"ra":np.array([82.88,299.43,350.28,187.07]),
         "decl":np.array([21.98,40.59,58.54,12.66])}

# Catalogue filter items
aart_coord = SkyCoord(aart.ra.values*u.deg, aart.decl.values*u.deg, frame='fk5')
ateam_coord = SkyCoord(ateam["ra"]*u.deg, ateam["decl"]*u.deg, frame='fk5')

delta = 100
vlssr_thresh= 5

In [13]:
pd.DataFrame({"DB":[],
                  "tot_candidate":[], 
                  "sigma_filter":[], 
                  "short_filter":[],
                  "ateam_filter":[], 
                  "aart_filter":[], 
                  "moving_filter":[], 
                  "remaining":[]}).to_csv("/data/survey/filter_stats.csv", index=False)

In [12]:
rm /data/survey/filter_stats.csv

In [9]:
db._configured = False
del db, session

In [ ]:
AS_databases = pd.read_csv("/home/kuiack/AS_databases.csv")["DBs"]

dataset = 1
logging.getLogger('sqlalchemy.engine').setLevel(query_loglevel)

for AS_db in AS_databases:
    


# get beginning and end time
    time_1 = time.time()
    db = tkp.db.Database(engine=engine, host=host, port=port,
                         user=user, password=password, database=AS_db)

    db.connect()
    session = db.Session()

    transients_query = """
        (SELECT taustart_ts 
        FROM image ORDER BY taustart_ts ASC LIMIT 30)
        UNION ALL
        (SELECT taustart_ts 
        FROM image ORDER BY taustart_ts DESC LIMIT 30)
    """

    time_range = pd.DataFrame(run_query(transients_query, AS_db, dataset,
                                          engine, host, port, user, password))

    db._configured = False
    del db, session

    starttime = np.min(time_range[np.abs(time_range.diff(periods= -10)
                                                                   .astype('timedelta64[s]')) < 15.]
                                                 .taustart_ts)
    endtime = np.max(time_range[np.abs(time_range.diff(periods= -10)
                                                                 .astype('timedelta64[s]')) < 15.]
                                               .taustart_ts)

    
    time_intervals = pd.date_range(start= starttime,
                                   end= endtime + datetime.timedelta(minutes=9), freq="10min")
    
    tot_time = len(pd.date_range(start=starttime,end=endtime, freq="1s"))

#     print "Get time intervals:",  time.time() - time_1

    print AS_db, starttime, endtime, len(time_intervals)

    tot_candidate = 0
    sigma_filter = 0
    short_filter = 0
    ateam_filter = 0
    aart_filter = 0
    moving_filter = 0
    remaining = 0
    count = 0
    for t1, t2 in zip(time_intervals[:-1],time_intervals[1:]):
        time_1 = time.time()
        db = tkp.db.Database(engine=engine, host=host, port=port,
                             user=user, password=password, database=AS_db)

        db.connect()
        session = db.Session()
    
        # Works, but is slow.
        transients_query = """
            SELECT  im.taustart_ts
                    ,im.freq_eff
                    ,ex.f_int
                    ,ex.f_int_err
                    ,ex.ra
                    ,ex.decl
                    ,ex.extract_type
                    ,ex.det_sigma
                    ,ax.runcat as runcatid
                    ,ax.f_datapoints
            FROM extractedsource ex
                 ,assocxtrsource ax
                 ,image im
                 ,runningcatalog rc
            WHERE ax.runcat = rc.id 
               AND ax.xtrsrc = ex.id
               AND ex.image = im.id
               AND rc.id IN
            (SELECT DISTINCT ns.runcat 
            FROM image im, newsource ns 
            WHERE ns.trigger_xtrsrc 
            IN (SELECT ex.id 
                FROM extractedsource ex, image im 
                WHERE ex.image = im.id 
                AND im.taustart_ts 
                BETWEEN '{}' AND '{}'))""".format(t1, t2)

        transients = pd.DataFrame(run_query(transients_query, AS_db, dataset,
                                  engine, host, port, user, password))    

        db._configured = False
        del db, session
        
        print "Run Query:", count,"/",len(time_intervals), t1, t2, time.time() - time_1
        count += 1
        if len(transients) == 0:
#             print "no runcats in time interval"
            continue

#         transients = transients[transients.rms_max < 3e2]




        # Reduce with conditions requireing simultaneous 5sigma and one 8sigma detection
        time_1 = time.time()
        N_detections = 0

        reduced = pd.DataFrame([])

        transients["round_times"] = transients.taustart_ts.dt.round("1s")
        
        for _id in np.unique(transients.runcatid):
            tot_candidate += 1
            _subTR = transients[(transients.runcatid == _id )]
            _subTR_extc = _subTR[(transients.extract_type == 0 )]

            if (np.max(_subTR.det_sigma) > 8 ) and  \
            (len(_subTR_extc[(transients.freq_eff >= 6e7 )]) > 0 ) and \
            (len(_subTR_extc[(transients.freq_eff <= 6e7 )]) > 0 ):
                

                # if the light curve is short min significant detection must at some point be simultaneous 
                # in both bands? ie there must be an overlap in 5sigma signal, would exclude 1 second dispersed transient. 
                if len(_subTR_extc) < 100:
                    diff_M = np.array(np.abs(_subTR_extc.taustart_ts.values-_subTR_extc.taustart_ts.values[:,None]))
                    diff_M[(diff_M == np.timedelta64(0,'ns'))] *= np.nan
           
                    if (np.min(diff_M) != np.timedelta64(1000000000,'ns')):
                          short_filter +=1
                          continue

                if len(reduced) == 0:
                    reduced = pd.DataFrame(_subTR)
                else:
                    reduced = pd.concat([reduced, _subTR])
            else:
                sigma_filter += 1

#         print "Sigma filter:", time.time() - time_1
        if len(reduced) == 0:
#             print "no valid transients in time interval"
            continue

    # Make small reference catalogue of candidates
        base = reduced.groupby("runcatid").median()
        base["taustart_ts"] = reduced.groupby("runcatid").first().taustart_ts
        base["timestep"] = [x.timestamp() for x in base.taustart_ts]

    # AARTFAAC Catalogue, Ateam, airplane filter 
        time_1 = time.time()


    #     filtered = pd.DataFrame([], columns=base.keys())
    #     AART_catsource = pd.DataFrame([], columns=base.keys())


        for i in base.index:
            try:
                c1 = SkyCoord(base.loc[i].ra*u.deg, base.loc[i].decl*u.deg, frame='fk5')

                c2 = SkyCoord(base.drop(index=i).ra.values*u.deg,
                          base.drop(index=i).decl.values*u.deg, frame='fk5')
            except IndexError:
                print i
            if np.min(c1.separation(ateam_coord).deg) < 10:
                    ateam_filter += 1
                    continue

            elif np.min(c1.separation(aart_coord).deg) < 3:
                aart_filter += 1
    #             if len(AART_catsource) == 0:
    #                 AART_catsource = pd.DataFrame(base.loc[i]).T
    #             else:
    #                 AART_catsource = pd.concat([AART_catsource, pd.DataFrame(base.loc[i]).T])


            elif not np.logical_or(((c1.separation(c2).deg) > 2.5),
                               (np.abs(base.loc[i].timestep - base.drop(index=i).timestep) > 600)).all():
                moving_filter += 1
#                 if len(filtered) == 0:
#                     filtered = pd.DataFrame(base.loc[i]).T
#                 else:
#                     filtered = pd.concat([filtered, pd.DataFrame(base.loc[i]).T])
            else:
                remaining += 1
#         print "Spacetime filter:", time.time() - time_1

#         if len(filtered) == 0:
#             print "no valid transients in time interval"
#             continue
    pd.DataFrame({"DB":[AS_db],
                  "tot_candidate":[tot_candidate], 
                  "sigma_filter":[sigma_filter],
                  "short_filter":[short_filter],
                  "ateam_filter":[ateam_filter], 
                  "aart_filter":[aart_filter], 
                  "moving_filter":[moving_filter], 
                  "remaining":[remaining] }).to_csv("/data/survey/filter_stats.csv",
                                                  mode="a", header=False, index=False)


ASr_201809230412 2018-09-23 04:12:11 2018-09-23 08:57:56 30
Run Query: 0 / 30 2018-09-23 04:12:11 2018-09-23 04:22:11 29.018419981


/afhome/kuiack/newtrap/lib/python2.7/site-packages/ipykernel_launcher.py:122: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
/afhome/kuiack/newtrap/lib/python2.7/site-packages/ipykernel_launcher.py:124: UserWarning: Boolean Series key will be reindexed to match DataFrame index.


Run Query: 1 / 30 2018-09-23 04:22:11 2018-09-23 04:32:11 10.7709419727
Run Query: 2 / 30 2018-09-23 04:32:11 2018-09-23 04:42:11 10.816131115
Run Query: 3 / 30 2018-09-23 04:42:11 2018-09-23 04:52:11 15.2404789925
Run Query: 4 / 30 2018-09-23 04:52:11 2018-09-23 05:02:11 11.3867230415
Run Query: 5 / 30 2018-09-23 05:02:11 2018-09-23 05:12:11 15.5278048515
Run Query: 6 / 30 2018-09-23 05:12:11 2018-09-23 05:22:11 12.204734087
Run Query: 7 / 30 2018-09-23 05:22:11 2018-09-23 05:32:11 27.1158969402
Run Query: 8 / 30 2018-09-23 05:32:11 2018-09-23 05:42:11 10.2482039928
Run Query: 9 / 30 2018-09-23 05:42:11 2018-09-23 05:52:11 11.3849649429
Run Query: 10 / 30 2018-09-23 05:52:11 2018-09-23 06:02:11 11.1052861214
Run Query: 11 / 30 2018-09-23 06:02:11 2018-09-23 06:12:11 10.4282441139
Run Query: 12 / 30 2018-09-23 06:12:11 2018-09-23 06:22:11 10.842550993
Run Query: 13 / 30 2018-09-23 06:22:11 2018-09-23 06:32:11 10.2343800068
Run Query: 14 / 30 2018-09-23 06:32:11 2018-09-23 06:42:11 10.8

Run Query: 15 / 49 2018-10-27 09:30:10 2018-10-27 09:40:10 9.57298898697
Run Query: 16 / 49 2018-10-27 09:40:10 2018-10-27 09:50:10 10.4373629093
Run Query: 17 / 49 2018-10-27 09:50:10 2018-10-27 10:00:10 13.6213879585
Run Query: 18 / 49 2018-10-27 10:00:10 2018-10-27 10:10:10 12.4592568874
Run Query: 19 / 49 2018-10-27 10:10:10 2018-10-27 10:20:10 15.1808729172
Run Query: 20 / 49 2018-10-27 10:20:10 2018-10-27 10:30:10 12.8968019485
Run Query: 21 / 49 2018-10-27 10:30:10 2018-10-27 10:40:10 12.4028749466
Run Query: 22 / 49 2018-10-27 10:40:10 2018-10-27 10:50:10 8.96112704277
Run Query: 23 / 49 2018-10-27 10:50:10 2018-10-27 11:00:10 12.312171936
Run Query: 24 / 49 2018-10-27 11:00:10 2018-10-27 11:10:10 10.6146168709
Run Query: 25 / 49 2018-10-27 11:10:10 2018-10-27 11:20:10 10.8011720181
Run Query: 26 / 49 2018-10-27 11:20:10 2018-10-27 11:30:10 8.43597006798
Run Query: 27 / 49 2018-10-27 11:30:10 2018-10-27 11:40:10 9.94458889961
Run Query: 28 / 49 2018-10-27 11:40:10 2018-10-27 11

Run Query: 20 / 49 2016-11-12 09:52:11.557000 2016-11-12 10:02:11.557000 12.5286729336
Run Query: 21 / 49 2016-11-12 10:02:11.557000 2016-11-12 10:12:11.557000 13.7106919289
Run Query: 22 / 49 2016-11-12 10:12:11.557000 2016-11-12 10:22:11.557000 12.8648819923
Run Query: 23 / 49 2016-11-12 10:22:11.557000 2016-11-12 10:32:11.557000 13.3467559814
Run Query: 24 / 49 2016-11-12 10:32:11.557000 2016-11-12 10:42:11.557000 12.5986368656
Run Query: 25 / 49 2016-11-12 10:42:11.557000 2016-11-12 10:52:11.557000 13.8616359234
Run Query: 26 / 49 2016-11-12 10:52:11.557000 2016-11-12 11:02:11.557000 12.8102180958
Run Query: 27 / 49 2016-11-12 11:02:11.557000 2016-11-12 11:12:11.557000 13.7085399628
Run Query: 28 / 49 2016-11-12 11:12:11.557000 2016-11-12 11:22:11.557000 12.4537088871
Run Query: 29 / 49 2016-11-12 11:22:11.557000 2016-11-12 11:32:11.557000 12.7139601707
Run Query: 30 / 49 2016-11-12 11:32:11.557000 2016-11-12 11:42:11.557000 17.5335488319
Run Query: 31 / 49 2016-11-12 11:42:11.5570

...

Run Query: 10 / 145 2019-01-13 01:42:00 2019-01-13 01:52:00 49.7353610992
Run Query: 11 / 145 2019-01-13 01:52:00 2019-01-13 02:02:00 55.8513820171
Run Query: 12 / 145 2019-01-13 02:02:00 2019-01-13 02:12:00 52.7056181431
Run Query: 13 / 145 2019-01-13 02:12:00 2019-01-13 02:22:00 53.3068950176
Run Query: 14 / 145 2019-01-13 02:22:00 2019-01-13 02:32:00 54.6707940102
Run Query: 15 / 145 2019-01-13 02:32:00 2019-01-13 02:42:00 51.4437611103
Run Query: 16 / 145 2019-01-13 02:42:00 2019-01-13 02:52:00 53.3540990353
Run Query: 17 / 145 2019-01-13 02:52:00 2019-01-13 03:02:00 51.4232738018
Run Query: 18 / 145 2019-01-13 03:02:00 2019-01-13 03:12:00 50.3628070354
Run Query: 19 / 145 2019-01-13 03:12:00 2019-01-13 03:22:00 56.45063591
Run Query: 20 / 145 2019-01-13 03:22:00 2019-01-13 03:32:00 51.9890592098
Run Query: 21 / 145 2019-01-13 03:32:00 2019-01-13 03:42:00 56.790733099
Run Query: 22 / 145 2019-01-13 03:42:00 2019-01-13 03:52:00 59.2248511314
Run Query: 23 / 145 2019-01-13 03:52:00 2

Run Query: 121 / 145 2019-01-13 20:12:00 2019-01-13 20:22:00 51.7762639523
Run Query: 122 / 145 2019-01-13 20:22:00 2019-01-13 20:32:00 53.3475849628
Run Query: 123 / 145 2019-01-13 20:32:00 2019-01-13 20:42:00 52.3954410553
Run Query: 124 / 145 2019-01-13 20:42:00 2019-01-13 20:52:00 49.1768109798
Run Query: 125 / 145 2019-01-13 20:52:00 2019-01-13 21:02:00 49.1172800064
Run Query: 126 / 145 2019-01-13 21:02:00 2019-01-13 21:12:00 50.199007988
Run Query: 127 / 145 2019-01-13 21:12:00 2019-01-13 21:22:00 49.9373250008
Run Query: 128 / 145 2019-01-13 21:22:00 2019-01-13 21:32:00 54.85085392
Run Query: 129 / 145 2019-01-13 21:32:00 2019-01-13 21:42:00 49.5302128792
Run Query: 130 / 145 2019-01-13 21:42:00 2019-01-13 21:52:00 49.2396919727
Run Query: 131 / 145 2019-01-13 21:52:00 2019-01-13 22:02:00 50.9707391262
Run Query: 132 / 145 2019-01-13 22:02:00 2019-01-13 22:12:00 48.1587011814
Run Query: 133 / 145 2019-01-13 22:12:00 2019-01-13 22:22:00 50.2474508286
Run Query: 134 / 145 2019-01

Run Query: 44 / 87 2019-01-19 14:42:00 2019-01-19 14:52:00 8.4103808403
Run Query: 45 / 87 2019-01-19 14:52:00 2019-01-19 15:02:00 7.20367479324
Run Query: 46 / 87 2019-01-19 15:02:00 2019-01-19 15:12:00 8.6559419632
Run Query: 47 / 87 2019-01-19 15:12:00 2019-01-19 15:22:00 7.11100292206
Run Query: 48 / 87 2019-01-19 15:22:00 2019-01-19 15:32:00 5.08457779884
Run Query: 49 / 87 2019-01-19 15:32:00 2019-01-19 15:42:00 4.98383188248
Run Query: 50 / 87 2019-01-19 15:42:00 2019-01-19 15:52:00 4.94917607307
Run Query: 51 / 87 2019-01-19 15:52:00 2019-01-19 16:02:00 4.97611904144
Run Query: 52 / 87 2019-01-19 16:02:00 2019-01-19 16:12:00 5.27422904968
Run Query: 53 / 87 2019-01-19 16:12:00 2019-01-19 16:22:00 5.1439909935
Run Query: 54 / 87 2019-01-19 16:22:00 2019-01-19 16:32:00 5.09547996521
Run Query: 55 / 87 2019-01-19 16:32:00 2019-01-19 16:42:00 5.15371608734
Run Query: 56 / 87 2019-01-19 16:42:00 2019-01-19 16:52:00 5.06477499008
Run Query: 57 / 87 2019-01-19 16:52:00 2019-01-19 17:0

Run Query: 22 / 37 2018-10-28 11:40:11 2018-10-28 11:50:11 6.98837399483
Run Query: 23 / 37 2018-10-28 11:50:11 2018-10-28 12:00:11 8.99849796295
Run Query: 24 / 37 2018-10-28 12:00:11 2018-10-28 12:10:11 5.94468307495
Run Query: 25 / 37 2018-10-28 12:10:11 2018-10-28 12:20:11 7.21434092522
Run Query: 26 / 37 2018-10-28 12:20:11 2018-10-28 12:30:11 6.85906004906
Run Query: 27 / 37 2018-10-28 12:30:11 2018-10-28 12:40:11 6.61122584343
Run Query: 28 / 37 2018-10-28 12:40:11 2018-10-28 12:50:11 6.95020794868
Run Query: 29 / 37 2018-10-28 12:50:11 2018-10-28 13:00:11 8.36998391151
Run Query: 30 / 37 2018-10-28 13:00:11 2018-10-28 13:10:11 6.99131989479
Run Query: 31 / 37 2018-10-28 13:10:11 2018-10-28 13:20:11 6.39964985847
Run Query: 32 / 37 2018-10-28 13:20:11 2018-10-28 13:30:11 6.39213109016
Run Query: 33 / 37 2018-10-28 13:30:11 2018-10-28 13:40:11 6.51300001144
Run Query: 34 / 37 2018-10-28 13:40:11 2018-10-28 13:50:11 6.95914697647
Run Query: 35 / 37 2018-10-28 13:50:11 2018-10-28 1

Run Query: 29 / 78 2019-01-10 11:53:00 2019-01-10 12:03:00 6.62410306931
Run Query: 30 / 78 2019-01-10 12:03:00 2019-01-10 12:13:00 3.31237792969
Run Query: 31 / 78 2019-01-10 12:13:00 2019-01-10 12:23:00 3.41664886475
Run Query: 32 / 78 2019-01-10 12:23:00 2019-01-10 12:33:00 3.35520291328
Run Query: 33 / 78 2019-01-10 12:33:00 2019-01-10 12:43:00 3.43925189972
Run Query: 34 / 78 2019-01-10 12:43:00 2019-01-10 12:53:00 4.86335897446
Run Query: 35 / 78 2019-01-10 12:53:00 2019-01-10 13:03:00 3.26772284508
Run Query: 36 / 78 2019-01-10 13:03:00 2019-01-10 13:13:00 3.35661005974
Run Query: 37 / 78 2019-01-10 13:13:00 2019-01-10 13:23:00 3.4326210022
Run Query: 38 / 78 2019-01-10 13:23:00 2019-01-10 13:33:00 3.47090697289
Run Query: 39 / 78 2019-01-10 13:33:00 2019-01-10 13:43:00 3.41212511063
Run Query: 40 / 78 2019-01-10 13:43:00 2019-01-10 13:53:00 3.54325604439
Run Query: 41 / 78 2019-01-10 13:53:00 2019-01-10 14:03:00 3.3563349247
Run Query: 42 / 78 2019-01-10 14:03:00 2019-01-10 14:

In [18]:
filter_stats = pd.read_csv("/data/survey/filter_stats.csv")

In [20]:
filter_stats.sum()

DB               ASr_201809230412ASr_201809280900ASr_2018092817...
aart_filter                                                  23070
ateam_filter                                                 12126
moving_filter                                                 6478
remaining                                                     9709
short_filter                                                 14468
sigma_filter                                                256720
tot_candidate                                               322571
dtype: object

In [25]:
print "tot sigma =", 256720+14468, (256720+14468.)/322571
print "tot catalogue =", 23070+12126, (23070+12126.)/322571
print "checksum", 256720+14468+9709+6478+12126+23070 - 322571

tot sigma = 271188 0.840707937167
tot catalogue = 35196 0.109110862415
checksum 0
